In [1]:
from tensorflow import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


In [12]:
from conformal import conformal
from conformal import utils
import torch

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../dataverse_files/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_last,name_first,race
841323,Torres,Jose,hispanic
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,Karam,MELINDA,nh_white
9156114,Brewer,LAIA,nh_black
...,...,...,...
3076722,Antunez Avila,Robert,hispanic
10023679,Davis,WYATT,nh_white
5846252,Scott,Jacquelyn,nh_white
5959131,Parton,Douglas,nh_white


In [4]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_ln_race.csv', columns=[])
rdf

,name_last
race,
asian,19431
hispanic,166865
nh_black,142675
nh_white,671029


In [5]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,9364
hispanic,41721
nh_black,23079
nh_white,145852


## Preprocessing the input data

In [6]:
sdf.head()

,name_last,name_first,race
841323,Torres,Jose,hispanic
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,Karam,MELINDA,nh_white
9156114,Brewer,LAIA,nh_black


In [7]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1166
Max feature len = 26, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [8]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


# Uncertainity Model

In [11]:
print('Build uncertainty model...')

#model_uncrtn = Sequential()
input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_)
output = Dense(num_classes, activation='softmax') (layer_)

model = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build uncertainty model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 32)            37312     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 120,260
Trainable params: 120,260
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15


2022-01-18 18:25:24.755922: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


22500/22500 [==============================] - 456s 20ms/step - loss: 0.6370 - accuracy: 0.7671 - val_loss: 0.6041 - val_accuracy: 0.7800
Epoch 2/15
22500/22500 [==============================] - 421s 19ms/step - loss: 0.5975 - accuracy: 0.7815 - val_loss: 0.5854 - val_accuracy: 0.7868
Epoch 3/15
22500/22500 [==============================] - 414s 18ms/step - loss: 0.5817 - accuracy: 0.7868 - val_loss: 0.5709 - val_accuracy: 0.7903
Epoch 4/15
22500/22500 [==============================] - 413s 18ms/step - loss: 0.5724 - accuracy: 0.7898 - val_loss: 0.5690 - val_accuracy: 0.7912
Epoch 5/15
22500/22500 [==============================] - 415s 18ms/step - loss: 0.5673 - accuracy: 0.7918 - val_loss: 0.5622 - val_accuracy: 0.7940
Epoch 6/15
22500/22500 [==============================] - 443s 20ms/step - loss: 0.5635 - accuracy: 0.7929 - val_loss: 0.5598 - val_accuracy: 0.7958
Epoch 7/15
22500/22500 [==============================] - 414s 18ms/step - loss: 0.5608 - accuracy: 0.7941 - val_loss

## Save model

In [13]:
model.save('../models/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [9]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')

In [10]:
svd_model = keras.models.load_model('../models/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [17]:
calib_loader = torch.utils.data.DataLoader((X_train, y_train),  shuffle=True)
val_loader = torch.utils.data.DataLoader((X_test,y_test), shuffle=True)    

In [21]:
X_train.shape

(800000, 20)

In [18]:
conf_model = conformal.ConformalModel(svd_model, calib_loader, alpha=0.1, kreg=5, lamda=0.01)

  0%|                                                                                                                                                                   | 0/2 [00:00<?, ?it/s]

Begin Platt scaling.
Computing logits for model (only happens once).


ValueError: not enough values to unpack (expected 2, got 1)